# Task 7: AutoFeatureSelector Tool
## This task is to test your understanding of various Feature Selection methods outlined in the lecture and the ability to apply this knowledge in a real-world dataset to select best features and also to build an automated feature selection tool as your toolkit

### Use your knowledge of different feature selector methods to build an Automatic Feature Selection tool
- Pearson Correlation
- Chi-Square
- RFE
- Embedded
- Tree (Random Forest)
- Tree (Light GBM)

### Dataset: FIFA 19 Player Skills
#### Attributes: FIFA 2019 players attributes like Age, Nationality, Overall, Potential, Club, Value, Wage, Preferred Foot, International Reputation, Weak Foot, Skill Moves, Work Rate, Position, Jersey Number, Joined, Loaned From, Contract Valid Until, Height, Weight, LS, ST, RS, LW, LF, CF, RF, RW, LAM, CAM, RAM, LM, LCM, CM, RCM, RM, LWB, LDM, CDM, RDM, RWB, LB, LCB, CB, RCB, RB, Crossing, Finishing, Heading, Accuracy, ShortPassing, Volleys, Dribbling, Curve, FKAccuracy, LongPassing, BallControl, Acceleration, SprintSpeed, Agility, Reactions, Balance, ShotPower, Jumping, Stamina, Strength, LongShots, Aggression, Interceptions, Positioning, Vision, Penalties, Composure, Marking, StandingTackle, SlidingTackle, GKDiving, GKHandling, GKKicking, GKPositioning, GKReflexes, and Release Clause.

In [140]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as ss
from collections import Counter
import math
from scipy import stats

In [141]:
player_df = pd.read_csv("https://raw.githubusercontent.com/pujan08/ML_AutoFeatureSelector/main/fifa19.csv")
print(player_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18207 entries, 0 to 18206
Data columns (total 89 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Unnamed: 0                18207 non-null  int64  
 1   ID                        18207 non-null  int64  
 2   Name                      18207 non-null  object 
 3   Age                       18207 non-null  int64  
 4   Photo                     18207 non-null  object 
 5   Nationality               18207 non-null  object 
 6   Flag                      18207 non-null  object 
 7   Overall                   18207 non-null  int64  
 8   Potential                 18207 non-null  int64  
 9   Club                      17966 non-null  object 
 10  Club Logo                 18207 non-null  object 
 11  Value                     18207 non-null  object 
 12  Wage                      18207 non-null  object 
 13  Special                   18207 non-null  int64  
 14  Prefer

In [142]:
numcols = ['Overall', 'Crossing','Finishing',  'ShortPassing',  'Dribbling','LongPassing', 'BallControl', 'Acceleration','SprintSpeed', 'Agility',  'Stamina','Volleys','FKAccuracy','Reactions','Balance','ShotPower','Strength','LongShots','Aggression','Interceptions']
catcols = ['Preferred Foot','Position','Body Type','Weak Foot']

In [143]:
player_df = player_df[numcols+catcols]

In [144]:
traindf = pd.concat([player_df[numcols], pd.get_dummies(player_df[catcols])],axis=1)
features = traindf.columns

traindf = traindf.dropna()

In [145]:
traindf = pd.DataFrame(traindf,columns=features)

In [146]:
y = traindf['Overall']>=87
X = traindf.copy()
del X['Overall']

In [147]:
X.head()

,Crossing,Finishing,ShortPassing,Dribbling,LongPassing,BallControl,Acceleration,SprintSpeed,Agility,Stamina,...,Body Type_Akinfenwa,Body Type_C. Ronaldo,Body Type_Courtois,Body Type_Lean,Body Type_Messi,Body Type_Neymar,Body Type_Normal,Body Type_PLAYER_BODY_TYPE_25,Body Type_Shaqiri,Body Type_Stocky
0,84.0,95.0,90.0,97.0,87.0,96.0,91.0,86.0,91.0,72.0,...,0,0,0,0,1,0,0,0,0,0
1,84.0,94.0,81.0,88.0,77.0,94.0,89.0,91.0,87.0,88.0,...,0,1,0,0,0,0,0,0,0,0
2,79.0,87.0,84.0,96.0,78.0,95.0,94.0,90.0,96.0,81.0,...,0,0,0,0,0,1,0,0,0,0
3,17.0,13.0,50.0,18.0,51.0,42.0,57.0,58.0,60.0,43.0,...,0,0,0,1,0,0,0,0,0,0
4,93.0,82.0,92.0,86.0,91.0,91.0,78.0,76.0,79.0,90.0,...,0,0,0,0,0,0,1,0,0,0


In [148]:
len(X.columns)

59

### Set some fixed set of features

In [149]:
feature_name = list(X.columns)
# no of maximum features we need to select
num_feats=30

## Filter Feature Selection - Pearson Correlation

### Pearson Correlation function

In [150]:
def cor_selector(X, y,num_feats):
    # Your code goes here (Multiple lines)
    cor_list = []
    feature_name = X.columns.tolist()

    for i in X.columns.tolist():
        cor = np.corrcoef(X[i], y)[0, 1]
        cor_list.append(cor)

    cor_list = [0 if np.isnan(i) else i for i in cor_list]
    cor_feature = X.iloc[:, np.argsort(np.abs(cor_list))[-num_feats:]].columns.tolist()
    cor_support = [True if i in cor_feature else False for i in feature_name]


    # Your code ends here
    return cor_support, cor_feature

In [151]:
cor_support, cor_feature = cor_selector(X, y, num_feats)
print(str(len(cor_feature)), 'selected features')

30 selected features


### List the selected features from Pearson Correlation

In [152]:
cor_feature

['Position_CM',
 'Interceptions',
 'Position_LW',
 'Balance',
 'Aggression',
 'Position_LAM',
 'Acceleration',
 'SprintSpeed',
 'Strength',
 'Stamina',
 'Weak Foot',
 'Agility',
 'Crossing',
 'Dribbling',
 'ShotPower',
 'LongShots',
 'Finishing',
 'BallControl',
 'FKAccuracy',
 'LongPassing',
 'Volleys',
 'ShortPassing',
 'Position_RF',
 'Position_LF',
 'Body Type_PLAYER_BODY_TYPE_25',
 'Body Type_Courtois',
 'Body Type_C. Ronaldo',
 'Body Type_Messi',
 'Body Type_Neymar',
 'Reactions']

## Filter Feature Selection - Chi-Sqaure

In [153]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler

### Chi-Squared Selector function

In [154]:
def chi_squared_selector(X, y, num_feats):
    # Your code goes here (Multiple lines)
    chi_selector = SelectKBest(chi2, k=num_feats)
    chi_selector.fit(X, y)
    chi_support = chi_selector.get_support()
    chi_feature = X.columns[chi_support]
    # Your code ends here
    return chi_support, chi_feature

In [155]:
chi_support, chi_feature = chi_squared_selector(X, y,num_feats)
print(str(len(chi_feature)), 'selected features')

30 selected features


### List the selected features from Chi-Square

In [156]:
chi_feature

Index(['Crossing', 'Finishing', 'ShortPassing', 'Dribbling', 'LongPassing',
       'BallControl', 'Acceleration', 'SprintSpeed', 'Agility', 'Stamina',
       'Volleys', 'FKAccuracy', 'Reactions', 'Balance', 'ShotPower',
       'Strength', 'LongShots', 'Aggression', 'Interceptions', 'Weak Foot',
       'Position_CM', 'Position_LAM', 'Position_LF', 'Position_LW',
       'Position_RF', 'Body Type_C. Ronaldo', 'Body Type_Courtois',
       'Body Type_Messi', 'Body Type_Neymar', 'Body Type_PLAYER_BODY_TYPE_25'],
      dtype='object')

## Wrapper Feature Selection - Recursive Feature Elimination

In [157]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler

### RFE Selector function

In [158]:
def rfe_selector(X, y, num_feats):
    # Your code goes here (Multiple lines)
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    estimator = LogisticRegression()
    rfe_selector = RFE(estimator, n_features_to_select=num_feats, step=1)
    rfe_selector = rfe_selector.fit(X_scaled, y)
    rfe_support = rfe_selector.support_
    rfe_feature = X.columns[rfe_support]
    # Your code ends here
    return rfe_support, rfe_feature

In [159]:
rfe_support, rfe_feature = rfe_selector(X, y,num_feats)
print(str(len(rfe_feature)), 'selected features')

30 selected features


### List the selected features from RFE

In [160]:
rfe_feature

Index(['Finishing', 'ShortPassing', 'LongPassing', 'BallControl',
       'Acceleration', 'SprintSpeed', 'Agility', 'Volleys', 'FKAccuracy',
       'Reactions', 'Strength', 'Weak Foot', 'Position_CAM', 'Position_CM',
       'Position_GK', 'Position_LAM', 'Position_LCB', 'Position_LF',
       'Position_LM', 'Position_LW', 'Position_RB', 'Position_RCB',
       'Position_RF', 'Position_RM', 'Position_RW', 'Body Type_Courtois',
       'Body Type_Lean', 'Body Type_Normal', 'Body Type_PLAYER_BODY_TYPE_25',
       'Body Type_Stocky'],
      dtype='object')

## Embedded Selection - Lasso: SelectFromModel

In [161]:
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler

In [162]:
def embedded_log_reg_selector(X, y, num_feats):
    # Your code goes here (Multiple lines)
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    estimator = LogisticRegression()
    embedded_lr_selector = SelectFromModel(estimator, max_features=num_feats)
    embedded_lr_selector.fit(X_scaled, y)
    embedded_lr_support = embedded_lr_selector.get_support()
    embedded_lr_feature = X.columns[embedded_lr_support]
    # Your code ends here
    return embedded_lr_support, embedded_lr_feature

In [163]:
embedded_lr_support, embedded_lr_feature = embedded_log_reg_selector(X, y, num_feats)
print(str(len(embedded_lr_feature)), 'selected features')

20 selected features


In [164]:
embedded_lr_feature

Index(['Finishing', 'ShortPassing', 'LongPassing', 'BallControl',
       'SprintSpeed', 'Agility', 'Volleys', 'Reactions', 'Strength',
       'Position_CAM', 'Position_CM', 'Position_GK', 'Position_LCB',
       'Position_LM', 'Position_RB', 'Position_RCB', 'Position_RW',
       'Body Type_Courtois', 'Body Type_Lean', 'Body Type_Stocky'],
      dtype='object')

## Tree based(Random Forest): SelectFromModel

In [165]:
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler

In [166]:
def embedded_rf_selector(X, y, num_feats):
    # Your code goes here (Multiple lines)
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    estimator = RandomForestClassifier(n_estimators=100, random_state=42)  # Adjust parameters as needed
    embedded_rf_selector = SelectFromModel(estimator, max_features=num_feats)
    embedded_rf_selector.fit(X_scaled, y)
    embedded_rf_support = embedded_rf_selector.get_support()
    embedded_rf_feature = X.columns[embedded_rf_support]
    # Your code ends here
    return embedded_rf_support, embedded_rf_feature

In [167]:
embedded_rf_support, embedded_rf_feature = embedded_rf_selector(X, y, num_feats)
print(str(len(embedded_rf_feature)), 'selected features')

18 selected features


In [168]:
embedded_rf_feature

Index(['Crossing', 'Finishing', 'ShortPassing', 'Dribbling', 'LongPassing',
       'BallControl', 'Acceleration', 'SprintSpeed', 'Agility', 'Stamina',
       'Volleys', 'FKAccuracy', 'Reactions', 'ShotPower', 'Strength',
       'LongShots', 'Aggression', 'Interceptions'],
      dtype='object')

## Tree based(Light GBM): SelectFromModel

In [169]:
from sklearn.feature_selection import SelectFromModel
from lightgbm import LGBMClassifier

In [170]:
def embedded_lgbm_selector(X, y, num_feats):
    # Your code goes here (Multiple lines)
    estimator = LGBMClassifier()
    estimator.fit(X, y)
    embedded_lgbm_selector = SelectFromModel(estimator, max_features=num_feats)
    embedded_lgbm_selector.fit(X, y)
    embedded_lgbm_support = embedded_lgbm_selector.get_support()
    embedded_lgbm_feature = X.columns[embedded_lgbm_support]
    # Your code ends here
    return embedded_lgbm_support, embedded_lgbm_feature

In [171]:
embedded_lgbm_support, embedded_lgbm_feature = embedded_lgbm_selector(X, y, num_feats)
print(str(len(embedded_lgbm_feature)), 'selected features')

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 55, number of negative: 18104
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002037 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1664
[LightGBM] [Info] Number of data points in the train set: 18159, number of used features: 50
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.003029 -> initscore=-5.796555
[LightGBM] [Info] Start training from score -5.796555
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best g

In [172]:
embedded_lgbm_feature

Index(['Crossing', 'Finishing', 'ShortPassing', 'Dribbling', 'LongPassing',
       'BallControl', 'Acceleration', 'SprintSpeed', 'Agility', 'Stamina',
       'Volleys', 'FKAccuracy', 'Reactions', 'Balance', 'ShotPower',
       'Strength', 'LongShots', 'Aggression', 'Interceptions'],
      dtype='object')

## Putting all of it together: AutoFeatureSelector Tool

In [173]:
pd.set_option('display.max_rows', None)
# put all selection together
feature_selection_df = pd.DataFrame({'Feature':feature_name, 'Pearson':cor_support, 'Chi-2':chi_support, 'RFE':rfe_support, 'Logistics':embedded_lr_support,
                                    'Random Forest':embedded_rf_support, 'LightGBM':embedded_lgbm_support})
# count the selected times for each feature
feature_selection_df['Total'] = np.sum(feature_selection_df, axis=1)
# display the top 100
feature_selection_df = feature_selection_df.sort_values(['Total','Feature'] , ascending=False)
feature_selection_df.index = range(1, len(feature_selection_df)+1)
feature_selection_df.head(num_feats)
print(feature_selection_df.head(num_feats))

                          Feature  Pearson  Chi-2    RFE  Logistics  \
1                         Volleys     True   True   True       True   
2                        Strength     True   True   True       True   
3                     SprintSpeed     True   True   True       True   
4                    ShortPassing     True   True   True       True   
5                       Reactions     True   True   True       True   
6                     LongPassing     True   True   True       True   
7                       Finishing     True   True   True       True   
8                     BallControl     True   True   True       True   
9                         Agility     True   True   True       True   
10                     FKAccuracy     True   True   True      False   
11                   Acceleration     True   True   True      False   
12                        Stamina     True   True  False      False   
13                      ShotPower     True   True  False      False   
14    

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:84: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  return reduction(axis=axis, out=out, **passkwargs)


## Can you build a Python script that takes dataset and a list of different feature selection methods that you want to try and output the best (maximum votes) features from all methods?

In [174]:
from sklearn.model_selection import train_test_split
def preprocess_dataset(dataset_path):
    # Your code starts here (Multiple lines)
    dataset = pd.read_csv(dataset_path)
    y = traindf['Overall']>=87
    X = traindf.copy()
    del X['Overall']
    X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2, random_state=42)

    # Choose the number of features you want to select (you can adjust this)
    num_feats = len(X.columns)


    # Your code ends here
    return X, y, num_feats

In [186]:
def autoFeatureSelector(dataset_path, methods=[]):
    # Parameters
    # data - dataset to be analyzed (csv file)
    # methods - various feature selection methods we outlined before, use them all here (list)

    # preprocessing
    X, y, num_feats = preprocess_dataset(dataset_path)

    # Run every method we outlined above from the methods list and collect returned best features from every method
    if 'pearson' in methods:
        cor_support, cor_feature = cor_selector(X, y,num_feats)
    if 'chi-square' in methods:
        chi_support, chi_feature = chi_squared_selector(X, y,num_feats)
    if 'rfe' in methods:
        rfe_support, rfe_feature = rfe_selector(X, y,num_feats)
    if 'log-reg' in methods:
        embedded_lr_support, embedded_lr_feature = embedded_log_reg_selector(X, y, num_feats)
    if 'rf' in methods:
        embedded_rf_support, embedded_rf_feature = embedded_rf_selector(X, y, num_feats)
    if 'lgbm' in methods:
        embedded_lgbm_support, embedded_lgbm_feature = embedded_lgbm_selector(X, y, num_feats)


    # Combine all the above feature list and count the maximum set of features that got selected by all methods
    #### Your Code starts here (Multiple lines)
    all_features = {
        'pearson': cor_feature if 'pearson' in methods else [],
        'chi-square': chi_feature if 'chi-square' in methods else [],
        'rfe': rfe_feature if 'rfe' in methods else [],
        'log-reg': embedded_lr_feature if 'log-reg' in methods else [],
        'rf': embedded_rf_feature if 'rf' in methods else [],
        'lgbm': embedded_lgbm_feature if 'lgbm' in methods else [],
    }
    common_features = list(all_features['pearson'])
    for method, features in all_features.items():
        common_features = [feat for feat in common_features if feat in features]

    best_features_df = pd.DataFrame({'Feature': common_features})

    # Select only the columns corresponding to the best features
    dataset_with_best_features = X[common_features]
    #### Your Code ends here
    return best_features_df, dataset_with_best_features

In [188]:
best_features, dataset_with_best_features = autoFeatureSelector(
    dataset_path="https://raw.githubusercontent.com/pujan08/ML_AutoFeatureSelector/main/fifa19.csv",
    methods=['pearson', 'chi-square', 'rfe', 'log-reg', 'rf', 'lgbm']
)

print("Best Features DataFrame:")
print(best_features)

print("\nDataset with Best Features:")
print(dataset_with_best_features.head(10))

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 55, number of negative: 18104
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001949 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1664
[LightGBM] [Info] Number of data points in the train set: 18159, number of used features: 50
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.003029 -> initscore=-5.796555
[LightGBM] [Info] Start training from score -5.796555
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best g

### Last, Can you turn this notebook into a python script, run it and submit the python (.py) file that takes dataset and list of methods as inputs and outputs the best features